<a href="https://colab.research.google.com/github/JayanthGuru/TeluguWordRecognizer/blob/main/final_Word_TOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Mounting the google drive
   
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# All the required imports

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from keras.models import load_model
from skimage.transform import rotate
import os
import shutil

#### Preprocessing Functions

In [ ]:
# Changing the directory

%cd '/content/drive/MyDrive/WordTOCR'

/content/drive/MyDrive/TOCR


In [ ]:
# Reads an image and returns an image as a 2d numpy array.
def read_image(path):
    '''
    Converting the image into numpy array.
    '''

    return np.asarray(Image.open(path).convert('L'))



# Binarizes an image.The input is a 2d numpy array.
def Binarize_image(img):
    '''
    Binarization is converting the image pixels to just have values 0 and 1. We have used Otsu's thresholding together with binary.
    '''

    # Apply Gaussian filtering 
    filtered_img = cv2.GaussianBlur(img,(5,5),0)
    # Apply thresholding.Otsu's and binary.
    ret,Thresh_img = cv2.threshold(filtered_img,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
    return Thresh_img

# Part - 1 Segmentation of Image

In [ ]:
# Segmentation Functions


# Function for distance
def split_lines_d(img):
    x=[]
    start = 0
    stop = 0
    for i in range(len(img)-1):
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i-1]==0) == 0 :
            start = i
        #if i!=len(img)-1:
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i+1]==0) == 0 :
            stop = i
            x.append(stop-start)      
    return x


def min_dist(img):
  splited_dist = split_lines_d(img)
  sort_dist=sorted(splited_dist)
  diff=[]
  for i in range(len(sort_dist)-1):
    z=sort_dist[i+1]-sort_dist[i]
    diff.append([z,sort_dist[i]])
  diff2=sorted(diff)
  if diff2[-1][0]<sort_dist[0]:
    dist=sort_dist[0]-2
  else:
    dist=diff2[-1][1]
  return dist


def split_lines_k(img):
    x=[]
    start = 0
    stop = 0
    for i in range(len(img)-1):
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i-1]==0) == 0 :
            start = i
        #if i!=len(img)-1:
        if np.count_nonzero(img[i]==0) > 0 and np.count_nonzero(img[i+1]==0) == 0 :
            stop = i
            x.append([stop,start])
    return x


def forming_lines(img):
  new=[]
  img2=split_lines_k(img)
  new.append([img2[0][1],img2[0][0]])
  for i in range(len(img2)-1):
    if img2[i+1][0]-img2[i+1][1]<=min_dist(img):
      if new[-1][0]==img2[i][1]:
        new[-1][1]=img2[i+1][0]
      else:
        new.append([img2[i][1],img2[i+1][0]])
    else:
      new.append([img2[i+1][1],img2[i+1][0]])
  return new
  

def split_lines(img):
  line_images=[]
  points=forming_lines(img)
  for i in points:
    start=i[0]
    stop=i[1]
    line_images.append(img[start:stop+1])
  return line_images


def split_line(img):
    # Outputs an array containing no. of zeroes in each row
    def num_zero(img):
        temp_list = []
        for i in range(len(img)):
            temp_list.append(np.count_nonzero(img[i]==0))
        return np.array(temp_list)
    
    # Image to be trimmed at the top and bottom. 
    # arr argument is the array containing no. of zeroes in each row
    def trim_image(img,arr):
        start = 0
        stop = 0
        for i in range(len(arr)):
            if arr[i]!=0:
                start = i
                break
        for i in reversed(range(len(arr))):
            if arr[i] != 0:
                stop = i
                break
        return (img[start:stop+1],arr[start:stop+1])

    
    # Returns length of consecutive zeroes
    def length(arr,ind,ele):
        if ind != len(arr)-1:
            i = ind
            while i<=len(arr)-1 and arr[i]==ele:
                i+=1
                
            if i != ind:
                return i-ind
            else:
                return 1
        else:
            return 


    # Returns True if no word exists after the current row.
    def EOL(arr,ind,ele):
        temp = True
        for i in range(ind,len(arr)):
            if arr[i] != 0:
                temp = False
        return temp


    # Returns the length above which the image should be split
    # The argument is the array of no. of zeroes
    def limiting_length(arr):
        temp_list = []
        i=0
        while i< len(arr):
            temp=0
            if arr[i] == 0:
                temp = length(arr,i,0)
                temp_list.append(temp)
            i=i+temp+1

        # maximum difference between consecutive elements
        temp_list.sort()
        diff = 0
        lim_len = 0
        for i in range(1,len(temp_list)):
            temp = temp_list[i] - temp_list[i-1]
            if temp > diff:
                diff = temp
                lim_len = temp_list[i-1]
        return lim_len
    
    img = np.transpose(img)
    zeroes = num_zero(img)
    trim_img, zeroes = trim_image(img,zeroes) 
    limit_len = limiting_length(zeroes)
    splited_images = []
    start = 0
    stop = 0
    i=1
    
    while i<len(zeroes):
        if zeroes[i] != 0:
            start = i
            while i < len(zeroes):
                if zeroes[i]!=0:
                    i+=1
                else:
                    if length(zeroes,i,0) > limit_len:
                        break
                    else:
                        i+=length(zeroes,i,0)
            if length(zeroes,i,0) > limit_len or EOL(zeroes,i,0):
                stop = i
                temp_img = np.transpose(trim_img[start:stop])
                splited_images.append(temp_img)
                i+=limit_len
        else:
            i+=1
    return splited_images


def split_word(img1):
    img = np.transpose(img1)
    splitted_letters = []
    i=0
    while i < len(img):
        if np.count_nonzero(img[i]==0) != 0:
            start = i
            j=i+1
            while j < len(img):
                if np.count_nonzero(img[j]==0) == 0:
                    break
                j+=1
            temp_img = np.transpose(img[start:j])
            splitted_letters.append(temp_img)
            i=j
        else:
            i+=1
    return splitted_letters

# PART-2 : PREDICTION OF CHARACTER

In [ ]:
# Load the models for prediction of character.
from keras.models import load_model
from keras.models import model_from_json
import json
with open('./Training/model_chars_tccnn-l.json') as infile:
    json_char = json.load(infile)

model_1 = model_from_json(json_char)
model_1.load_weights('./Training/model_chars_tccnn-l.hdf5')

with open('./Training/model_v_g.json') as infile:
    json_char = json.load(infile)

model_2 = model_from_json(json_char)
model_2.load_weights('./Training/model_v_g_weights.hdf5')

In [ ]:
#FUNCTION
def predict_from_char(char):
  '''
  Takes the input numpy array of image and return a index of character and index of vattu.
  '''

  img=char
  #img=cv2.transpose(img)
  img=cv2.resize(img,(32,32))
  img = np.asarray(img).reshape(1, 1, 32, 32)
  img = img.astype('float32')
  img = img/255.0
  chars=[]
  vg=[]
  out =  model_1.predict(img)
  chars.append(np.where(out==out.max())[1][:]+1)
  if np.where(out==out.max())[1][:]+1>=20 and np.where(out==out.max())[1][:]+1<=55:
    out2 = model_2.predict(img)
    vg.append(np.where(out2==out2.max())[1][:])
  else:
    vg.append(np.array([-1]))
  return chars[0][0],vg[0][0]

In [ ]:
# Recognizing the final-dataset images from handwritten_telugu_word_dataset.


import difflib
import os


# loading the char.txt and vattu_gunintam.txt files 

f1 = open('char.txt','r')
temp_list = f1.readlines()
f1.close()
char = {}
i=0
for i in range(len(temp_list)):
    if i%2 == 0:
        char[(i/2)+1] = temp_list[i+1] 

f2 = open('vattu_gunintam.txt','r')
temp_list = f2.readlines()
f2.close()
vattu_gunintam={}
i=0
for i in range(len(temp_list)):
    if i%2 == 0:
        vattu_gunintam[(i/2)] = temp_list[i+1]



predicted_words = []
lengthOfDataset = len(os.listdir("/content/drive/MyDrive/handwritten_telugu_word_dataset/final"))
failed_cases = []



startIdx = 1
endIdx = 3000
for i in range(startIdx, endIdx+1):
    img = read_image(f'/content/drive/MyDrive/handwritten_telugu_word_dataset/final/{i}.jpg')
    img = Binarize_image(img)


    recognized_word = ""
    try:
      for line in split_lines(img):
          for word in split_line(line):
              for letter in split_word(word):
                  a,b = predict_from_char(letter)
                  if b==-1:
                      s = (char[a][:-2])
                  else:
                      s = (char[a][:-2]+vattu_gunintam[b][:-2])
                  l = s.split("&#")
                  for i in l[1:]:
                    recognized_word += chr(int(i[:4]))

      predicted_words.append(recognized_word)
    except Exception:
      failed_cases.append(i)
      continue    




Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 22ms/step


134

In [ ]:


import os
import random

corpus = set()
original_words = []

final_details = open("/content/drive/MyDrive/WordTOCR/final_details.txt","w")          #this is for ouput file
final_details.write("predicted_words length "+ str(len(predicted_words))+"\n")

final_details.write("no of failed_cases ="+str(len(failed_cases))+"\n")
final_details.write("failed cases "+str(failed_cases)+"\n")


# Creating a Corpus using final.txt
corpus_path = "/content/drive/MyDrive/handwritten_telugu_word_dataset/final.txt"
idx = 0
for i in open(corpus_path, "r"):
  i = i.replace("\n","")
  i = i.strip()
  corpus.add(i)
  if(idx not in failed_cases):
    original_words.append(i)
  idx += 1


# loading Corpus file.
corpus = list(corpus)
corpus_file = open("/content/drive/MyDrive/handwritten_telugu_word_dataset/corpus.txt","w")
for i in corpus:
  corpus_file.write(i+"\n")
corpus_file.close()

random.shuffle(corpus)

final_result = open("final_result.txt","w")
final_result.write("dataset length "+str(lengthOfDataset - len(failed_cases))+"\n")


20

In [ ]:
import shutil


# Getcloase method.  
recognized_words = []  
for word in predicted_words:
    matchs = difflib.get_close_matches(word, corpus, cutoff = 0.4)
    if(len(matchs) != 0):
      closest_match = matchs[0]
    else:
      closest_match = corpus[-1]
    recognized_words.append(closest_match)


# writing the result of analysis to final_result.txt 
final_result.write("recognized_words length" +str(len(recognized_words)))
final_result.write("\n")
print("recognized_words length" +str(len(recognized_words)))

word_error_count = 0 
char_error_count = 0
total_words = 0
total_chars = 0
accurate_words = {}
non_accurate_words = {}

pred_char_error_count = 0
pred_word_error_count = 0


final_result.write("len of predicted_words  "+str(len(predicted_words)))
final_result.write("\n")
print("len of predicted_words  "+str(len(predicted_words)))
final_result.write("len of recognized_words  "+str(len(recognized_words)))
final_result.write("\n")
print("len of recognized_words  "+str(len(recognized_words)))
final_result.write("len of original_words  "+str(len(original_words)))
final_result.write("\n")
print("len of original_words  "+str(len(original_words)))


final_details.write("PREDICTED  |   Recognized   |  ACTUAL\n")

for i in range(lengthOfDataset - len(failed_cases)):
  final_details.write(str(i+1)+"   |    "+predicted_words[i]+"   |   "+recognized_words[i]+"  |  "+original_words[i]+"\n")
  total_words += 1
  total_chars += len(original_words[i])
  if(recognized_words[i] != original_words[i]):
    word_error_count += 1
    for j in range(min(len(original_words[i]), len(recognized_words[i]))):
      if(original_words[i][j] != recognized_words[i][j]):
        char_error_count += 1
    char_error_count += abs(len(recognized_words[i]) - j)  
    non_accurate_words[i+1] = original_words[i]
  else:
    accurate_words[i+1] = original_words[i]


#  -------------------------------------
for i in range(lengthOfDataset - len(failed_cases)):
  if(predicted_words[i] != original_words[i]):
    pred_word_error_count += 1
    for j in range(min(len(original_words[i]), len(predicted_words[i]))):
      if(original_words[i][j] != predicted_words[i][j]):
        pred_char_error_count += 1
    pred_char_error_count += abs(len(predicted_words[i]) - j)  


final_result.write("Only with Predicted : \n")
print("Only with Predicted : ")
final_result.write("pred_word_error_count : "+str(pred_word_error_count)+"\n")
print("pred_word_error_count : "+str(pred_word_error_count))
final_result.write("pred_char_error_count : "+str(pred_char_error_count)+"\n\n\n")
print("pred_char_error_count : "+str(pred_char_error_count))
final_result.write("Predicted CER : "+str((pred_char_error_count//total_chars)*100 )+"\n")
print("Predicted CER : "+str((pred_char_error_count/total_chars)*100 ))
final_result.write("Predicted WER : "+str((pred_word_error_count//total_words)*100))
print("Predicted WER : "+str((pred_word_error_count/total_words)*100))
#  -------------------------------------


final_result.write("GET close method : \n")
final_result.write("word error count  "+str(word_error_count))
final_result.write("\n")
print("word error count  "+str(word_error_count))
final_result.write("char error count " + str(char_error_count))
final_result.write("\n")
print("char error count " + str(char_error_count))
final_result.write("total words " + str(total_words))
final_result.write("\n")
print("total words " + str(total_words))
final_result.write("total chars " + str(total_chars))
final_result.write("\n")
print("total chars " + str(total_chars))


WER = (word_error_count/total_words) * 100
CER = (char_error_count/total_chars) * 100

final_result.write("WER : " +str(WER))
final_result.write("\n")
print("WER : " +str(WER))
final_result.write("CER : " +str(CER))
final_result.write("\n")
print("CER : " +str(CER))
final_result.write("Word Accuracy : "+ str(100 - WER))
final_result.write("\n")
print("Word Accuracy : "+ str(100 - WER))
final_result.write("Char Accuracy : "+ str(100 - CER))
final_result.write("\n")
print("Char Accuracy : "+ str(100 - CER))


final_details.write("\nCorrectly identified \n")
for i in accurate_words:
  final_details.write(str(i)+", "+str(accurate_words[i])+"\n")

final_details.write("\nIncorrectly identified words\n")
for i in non_accurate_words:
  final_details.write(str(i)+", "+str(non_accurate_words[i])+"\n")

recognized_words length2980
len of predicted_words  2980
len of recognized_words  2980
len of original_words  2980
Only with Predicted : 
pred_word_error_count : 2603
pred_char_error_count : 14692
Predicted CER : 107.45264389673079
Predicted WER : 87.3489932885906
word error count  1668
char error count 8134
total words 2980
total chars 13673
WER : 55.97315436241611
CER : 59.4895048635998
Word Accuracy : 44.02684563758389
Char Accuracy : 40.5104951364002


In [ ]:
# Levenshtein distance method with allowed operation = 1


!pip install Levenshtein


from Levenshtein import distance

def fuzzy_match(query, options, max_distance):
    matches = []
    for option in options:
      if distance(query, option) <= max_distance:
        matches.append(option)
    if len(matches) > 0:
      return matches
    return ""



recognized_words = []  
for word in predicted_words:
    matchs = fuzzy_match(word, corpus, 1)
    if(len(matchs) != 0):
      closest_match = matchs[0]
    else:
      closest_match = corpus[-1]
    recognized_words.append(closest_match)


final_result.write("recognized_words length" +str(len(recognized_words)))
final_result.write("\n")
print("recognized_words length" +str(len(recognized_words)))

word_error_count = 0 
char_error_count = 0
total_words = 0
total_chars = 0
accurate_words = {}
non_accurate_words = {}


final_result.write("len of predicted_words  "+str(len(predicted_words)))
final_result.write("\n")
print("len of predicted_words  "+str(len(predicted_words)))
final_result.write("len of recognized_words  "+str(len(recognized_words)))
final_result.write("\n")
print("len of recognized_words  "+str(len(recognized_words)))
final_result.write("len of original_words  "+str(len(original_words)))
final_result.write("\n")
print("len of original_words  "+str(len(original_words)))


# final_details.write("PREDICTED  |   Recognized   |  ACTUAL\n")

for i in range(lengthOfDataset - len(failed_cases)):
  # print(str(i+1)+"   |    "+predicted_words[i]+"   |   "+recognized_words[i]+"  |  "+original_words[i])
  total_words += 1
  total_chars += len(original_words[i])
  if(recognized_words[i] != original_words[i]):
    word_error_count += 1
    for j in range(min(len(original_words[i]), len(recognized_words[i]))):
      if(original_words[i][j] != recognized_words[i][j]):
        char_error_count += 1
    char_error_count += abs(len(recognized_words[i]) - j)  
    # non_accurate_words[i+1] = original_words[i]
  # else:
  #   accurate_words[i+1] = original_words[i]


final_result.write("AUTO CORRECT 1 : \n")
final_result.write("word error count  "+str(word_error_count))
final_result.write("\n")
print("word error count  "+str(word_error_count))
final_result.write("char error count " + str(char_error_count))
final_result.write("\n")
print("char error count " + str(char_error_count))
final_result.write("total words " + str(total_words))
final_result.write("\n")
print("total words " + str(total_words))
final_result.write("total chars " + str(total_chars))
final_result.write("\n")
print("total chars " + str(total_chars))


WER = (word_error_count/total_words) * 100
CER = (char_error_count/total_chars) * 100

final_result.write("WER : " +str(WER))
final_result.write("\n")
print("WER : " +str(WER))
final_result.write("CER : " +str(CER))
final_result.write("\n")
print("CER : " +str(CER))
final_result.write("Word Accuracy : "+ str(100 - WER))
final_result.write("\n")
print("Word Accuracy : "+ str(100 - WER))
final_result.write("Char Accuracy : "+ str(100 - CER))
final_result.write("\n")
print("Char Accuracy : "+ str(100 - CER))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.1 MB/s eta 0:00:00
recognized_words length2980
len of predicted_words  2980
len of recognized_words  2980
len of original_words  2980
word error count  2322
char error count 8347
total words 2980
total chars 13673
WER : 77.91946308724832
CER : 61.047319534849706
Word Accuracy : 22.080536912751683
Char Accuracy : 38.952680465150294


In [ ]:
# Levenshtein distance method with allowed operation = 2


recognized_words = []  
for word in predicted_words:
    matchs = fuzzy_match(word, corpus, 2)
    if(len(matchs) != 0):
      closest_match = matchs[0]
    else:
      closest_match = corpus[-1]
    recognized_words.append(closest_match)


final_result.write("recognized_words length" +str(len(recognized_words)))
final_result.write("\n")
print("recognized_words length" +str(len(recognized_words)))

word_error_count = 0 
char_error_count = 0
total_words = 0
total_chars = 0
accurate_words = {}
non_accurate_words = {}


final_result.write("len of predicted_words  "+str(len(predicted_words)))
final_result.write("\n")
print("len of predicted_words  "+str(len(predicted_words)))
final_result.write("len of recognized_words  "+str(len(recognized_words)))
final_result.write("\n")
print("len of recognized_words  "+str(len(recognized_words)))
final_result.write("len of original_words  "+str(len(original_words)))
final_result.write("\n")
print("len of original_words  "+str(len(original_words)))


# final_details.write("PREDICTED  |   Recognized   |  ACTUAL\n")

for i in range(lengthOfDataset - len(failed_cases)):
  # print(str(i+1)+"   |    "+predicted_words[i]+"   |   "+recognized_words[i]+"  |  "+original_words[i])
  total_words += 1
  total_chars += len(original_words[i])
  if(recognized_words[i] != original_words[i]):
    word_error_count += 1
    for j in range(min(len(original_words[i]), len(recognized_words[i]))):
      if(original_words[i][j] != recognized_words[i][j]):
        char_error_count += 1
    char_error_count += abs(len(recognized_words[i]) - j)  
    # non_accurate_words[i+1] = original_words[i]
  # else:
  #   accurate_words[i+1] = original_words[i]


final_result.write("AUTO CORRECT 2 : \n")
final_result.write("word error count  "+str(word_error_count))
final_result.write("\n")
print("word error count  "+str(word_error_count))
final_result.write("char error count " + str(char_error_count))
final_result.write("\n")
print("char error count " + str(char_error_count))
final_result.write("total words " + str(total_words))
final_result.write("\n")
print("total words " + str(total_words))
final_result.write("total chars " + str(total_chars))
final_result.write("\n")
print("total chars " + str(total_chars))


WER = (word_error_count/total_words) * 100
CER = (char_error_count/total_chars) * 100

final_result.write("WER : " +str(WER))
final_result.write("\n")
print("WER : " +str(WER))
final_result.write("CER : " +str(CER))
final_result.write("\n")
print("CER : " +str(CER))
final_result.write("Word Accuracy : "+ str(100 - WER))
final_result.write("\n")
print("Word Accuracy : "+ str(100 - WER))
final_result.write("Char Accuracy : "+ str(100 - CER))
final_result.write("\n")
print("Char Accuracy : "+ str(100 - CER))

recognized_words length2980
len of predicted_words  2980
len of recognized_words  2980
len of original_words  2980
word error count  2567
char error count 9465
total words 2980
total chars 13673
WER : 86.14093959731544
CER : 69.22401813793608
Word Accuracy : 13.859060402684563
Char Accuracy : 30.775981862063915


In [ ]:
# Levenshtein distance method with allowed operation = 3


recognized_words = []  
for word in predicted_words:
    matchs = fuzzy_match(word, corpus, 3)
    if(len(matchs) != 0):
      closest_match = matchs[0]
    else:
      closest_match = corpus[-1]
    recognized_words.append(closest_match)


final_result.write("recognized_words length" +str(len(recognized_words)))
final_result.write("\n")
print("recognized_words length" +str(len(recognized_words)))

word_error_count = 0 
char_error_count = 0
total_words = 0
total_chars = 0
accurate_words = {}
non_accurate_words = {}


final_result.write("len of predicted_words  "+str(len(predicted_words)))
final_result.write("\n")
print("len of predicted_words  "+str(len(predicted_words)))
final_result.write("len of recognized_words  "+str(len(recognized_words)))
final_result.write("\n")
print("len of recognized_words  "+str(len(recognized_words)))
final_result.write("len of original_words  "+str(len(original_words)))
final_result.write("\n")
print("len of original_words  "+str(len(original_words)))


# final_details.write("PREDICTED  |   Recognized   |  ACTUAL\n")

for i in range(lengthOfDataset - len(failed_cases)):
  # print(str(i+1)+"   |    "+predicted_words[i]+"   |   "+recognized_words[i]+"  |  "+original_words[i])
  total_words += 1
  total_chars += len(original_words[i])
  if(recognized_words[i] != original_words[i]):
    word_error_count += 1
    for j in range(min(len(original_words[i]), len(recognized_words[i]))):
      if(original_words[i][j] != recognized_words[i][j]):
        char_error_count += 1
    char_error_count += abs(len(recognized_words[i]) - j)  
    # non_accurate_words[i+1] = original_words[i]
  # else:
  #   accurate_words[i+1] = original_words[i]


final_result.write("AUTO CORRECT 3 : \n")
final_result.write("word error count  "+str(word_error_count))
final_result.write("\n")
print("word error count  "+str(word_error_count))
final_result.write("char error count " + str(char_error_count))
final_result.write("\n")
print("char error count " + str(char_error_count))
final_result.write("total words " + str(total_words))
final_result.write("\n")
print("total words " + str(total_words))
final_result.write("total chars " + str(total_chars))
final_result.write("\n")
print("total chars " + str(total_chars))


WER = (word_error_count/total_words) * 100
CER = (char_error_count/total_chars) * 100

final_result.write("WER : " +str(WER))
final_result.write("\n")
print("WER : " +str(WER))
final_result.write("CER : " +str(CER))
final_result.write("\n")
print("CER : " +str(CER))
final_result.write("Word Accuracy : "+ str(100 - WER))
final_result.write("\n")
print("Word Accuracy : "+ str(100 - WER))
final_result.write("Char Accuracy : "+ str(100 - CER))
final_result.write("\n")
print("Char Accuracy : "+ str(100 - CER))


final_result.close()
final_details.close()

recognized_words length2980
len of predicted_words  2980
len of recognized_words  2980
len of original_words  2980
word error count  2730
char error count 11405
total words 2980
total chars 13673
WER : 91.61073825503355
CER : 83.41256490894463
Word Accuracy : 8.389261744966447
Char Accuracy : 16.58743509105537
